# Advanced pyALF part 1: Running ALF

In [1]:
from pprint import pprint  # Pretty print
from py_alf import ALF_source, Simulation  # Interface with ALF

## ALF_source
Create instance of `ALF_source`, choosing to checkout the git branch `'211-add-nematic-dirac-hamiltonian'`, since the Nematic Dirac Hamiltonian is not on master.

Reminder: Directory containing the ALF code is taken from environment variable `$ALF_DIR`, if present.

In [2]:
alf_src = ALF_source(
    branch='211-add-nematic-dirac-hamiltonian',
)

Checking out branch 211-add-nematic-dirac-hamiltonian
Your branch is up to date with 'origin/211-add-nematic-dirac-hamiltonian'.


Already on '211-add-nematic-dirac-hamiltonian'


Check available Hamiltonians:

In [3]:
alf_src.get_ham_names()

['Kondo',
 'Hubbard',
 'Hubbard_Plain_Vanilla',
 'tV',
 'LRC',
 'Z2_Matter',
 'Nematic_Dirac']

Print valid parameters and their defaults for Nematic Dirac Hamiltonian:

In [4]:
pprint(alf_src.get_default_params('Nematic_Dirac', include_generic=False))

OrderedDict([('VAR_Nematic_Dirac',
              {'Global_J': {'comment': 'J for proposing global updates',
                            'defined_in_base': False,
                            'value': 1.0},
               'Global_h': {'comment': 'h for proposing global updates',
                            'defined_in_base': False,
                            'value': 3.0},
               'Global_type': {'comment': 'Type of global update. Possible '
                                          "values: 'Wolff', 'Geo', 'switch', "
                                          "'flip'",
                               'defined_in_base': False,
                               'value': ''},
               'Ham_J': {'comment': 'Ferromagnetic Ising interaction',
                         'defined_in_base': False,
                         'value': 1.0},
               'Ham_chem': {'comment': 'Chemical potential',
                            'defined_in_base': False,
                            'value': 0

## Parallel Tempering
The simulation object below shows an example of Parallel Tempering, which runs simulations with different parameters in parallel and exchanges configurations between them. The method has been developed to address ergodicity issues, but in this case it's only used to execute a parameter sweep "in one go". For more techincal details on Parallel Tempering, refer to the ALF documentation.

Parallel Tempering is enabled by making `sim_dict` a list of dictionaries, instead of a single dictionary.

In [5]:
for L, time in zip([4, 6, 8, 10], [.2, 1., 7., 18.]):
    print(f'====== L={L} ======')
    sim = Simulation(
        alf_src,
        'Nematic_Dirac',
        [{
            # Model specific parameters
            'Model_vers': 1,
            'L1': L,
            'L2': L,
            'beta': L*4.,
            'Ham_xi': 0.25,
            'Ham_h': h,
            # QMC parameters
            'Ltau': 1,
            'CPU_MAX': time,
            'NSweep': 20,
            'mpi_per_parameter_set': 1,
            'Tempering_calc_det': False,
        } for h in [2.5, 3.0, 3.5, 4.0]],
        machine='intel',
        mpi=True,
        n_mpi=4,
    )
    if L == 4:
        # We only need to compile once
        sim.compile()
    sim.run()
    sim.print_info_file()

====== L=4 ======
Compiling ALF... 
Cleaning up Prog/
Cleaning up Libraries/
Cleaning up Analysis/
Compiling Libraries


ar: creating modules_90.a
ar: creating libqrref.a
ana_mod.F90(1213): warning #5268: Extension to standard: The text exceeds right hand column allowed on the line.
            Write(error_unit,*) 'Evaluating the mutual information between A and B requires the entanglement entropies of A, B and the union of A and B, i.e. Nobs=4 (3 + 1 for the phase)'
-------------------------------------------------------------------------------------------------------------------------------------^


Compiling Analysis


ifort: command line remark #10412: option '-mkl' is deprecated and will be removed in a future release. Please use the replacement option '-qmkl'
ifort: command line remark #10412: option '-mkl' is deprecated and will be removed in a future release. Please use the replacement option '-qmkl'
ifort: command line remark #10412: option '-mkl' is deprecated and will be removed in a future release. Please use the replacement option '-qmkl'
Max_SAC.F90(164): warning #5268: Extension to standard: The text exceeds right hand column allowed on the line.
       If (  nbin_qmc < 2*Ntau .and. N_cov == 1  )   Write(50,*) 'You do not seem to have enough bins for a reliable estimate of the covariance '
-------------------------------------------------------------------------------------------------------------------------------------^
ifort: command line remark #10412: option '-mkl' is deprecated and will be removed in a future release. Please use the replacement option '-qmkl'
ifort: command line rem

Compiling Program
Parsing Hamiltonian parameters
filename: Hamiltonians/Hamiltonian_Kondo_smod.F90
filename: Hamiltonians/Hamiltonian_Hubbard_smod.F90
filename: Hamiltonians/Hamiltonian_Hubbard_Plain_Vanilla_smod.F90
filename: Hamiltonians/Hamiltonian_tV_smod.F90
filename: Hamiltonians/Hamiltonian_LRC_smod.F90
filename: Hamiltonians/Hamiltonian_Z2_Matter_smod.F90
filename: Hamiltonians/Hamiltonian_Nematic_Dirac_smod.F90
Compiling program modules


Operator_mod.F90(143): warning #5268: Extension to standard: The text exceeds right hand column allowed on the line.
!> * On exit:  phase of  \f$ W(C)  =   \left[ \left( \prod_{n,\tau,f}  \exp \left[ g_f(n) \alpha_f(n) \phi(\sigma(n,\tau)) \right] \right) \det(M_f(C))\right]^{N_{SUN}} \prod_{n,\tau }\gamma(\sigma(n,\tau)) \f$
------------------------------------------------------------------------------------------------------------------------------------^
Predefined_Obs_mod.F90(347): warning #5268: Extension to standard: The text exceeds right hand column allowed on the line.
!>  Returns:  \frac{2N}{N^2-1}\sum_{a=1}{N^2 - 1 }  <c^{dag}_i(tau) T^a c_i (tau)  c^{dag}_j T^a  c_j>  where  T^a are th generators of SU(N)
------------------------------------------------------------------------------------------------------------------------------------^
./Cotunneling_dimer_obs.F90(226): warning #5268: Extension to standard: The text exceeds right hand column allowed on the line.
           

Link program


ifort: command line remark #10412: option '-mkl' is deprecated and will be removed in a future release. Please use the replacement option '-qmkl'


Done.
Prepare directory "/scratch/pyalf-docu/doc/source/examples/ALF_data/temper_Nematic_Dirac_Model_vers=1_L1=4_L2=4_beta=16.0_xi=0.25_h=2.5" for Monte Carlo run.
Create new directory.
Prepare directory "/scratch/pyalf-docu/doc/source/examples/ALF_data/temper_Nematic_Dirac_Model_vers=1_L1=4_L2=4_beta=16.0_xi=0.25_h=2.5/Temp_0" for Monte Carlo run.
Create new directory.
Prepare directory "/scratch/pyalf-docu/doc/source/examples/ALF_data/temper_Nematic_Dirac_Model_vers=1_L1=4_L2=4_beta=16.0_xi=0.25_h=2.5/Temp_1" for Monte Carlo run.
Create new directory.
Prepare directory "/scratch/pyalf-docu/doc/source/examples/ALF_data/temper_Nematic_Dirac_Model_vers=1_L1=4_L2=4_beta=16.0_xi=0.25_h=2.5/Temp_2" for Monte Carlo run.
Create new directory.
Prepare directory "/scratch/pyalf-docu/doc/source/examples/ALF_data/temper_Nematic_Dirac_Model_vers=1_L1=4_L2=4_beta=16.0_xi=0.25_h=2.5/Temp_3" for Monte Carlo run.
Create new directory.
Run /home/jschwab/Programs/ALF/Prog/ALF.out
 ALF Copyright (C) 201

Check info files produced by ALF. It is strongly advised to take a look at these after finished runs, in particular "Precision Green" and "Precision Phase". These should be small, of order $10^{-8}$ or smaller. If they're bigger, one should decrease the stabilization intervall `Nwrap` (see parameter list `'VAR_QMC'` above). In our case, they're of order $10^{-14}$ and one might consider increasing `Nwrap` to speed up the simulation.